<a href="https://colab.research.google.com/github/thimmareddyhd28/data-science-and-analytics/blob/main/Copy_of_Untitled7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "offline"
os.environ["HF_DISABLE_WANDB"] = "1"


In [ ]:
import os
print(os.listdir("/content"))


['.config', '.ipynb_checkpoints', 'data[1].txt', 'sample_data']


In [ ]:
!pip install transformers datasets sentencepiece torch


In [ ]:
file_path = "/content/data[1].txt"

with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
    text = f.read()

print(text[:2000])


Chapter I

IN WHICH PHILEAS FOGG AND PASSEPARTOUT ACCEPT EACH OTHER,
THE ONE AS MASTER, THE OTHER AS MAN


Mr. Phileas Fogg lived, in 1872, at No. 7, Saville Row, Burlington
Gardens, the house in which Sheridan died in 1814.  He was one of the
most noticeable members of the Reform Club, though he seemed always to
avoid attracting attention; an enigmatical personage, about whom little
was known, except that he was a polished man of the world.  People said
that he resembled Byron--at least that his head was Byronic; but he was
a bearded, tranquil Byron, who might live on a thousand years without
growing old.

Certainly an Englishman, it was more doubtful whether Phileas Fogg was
a Londoner.  He was never seen on 'Change, nor at the Bank, nor in the
counting-rooms of the "City"; no ships ever came into London docks of
which he was the owner; he had no public employment; he had never been
entered at any of the Inns of Court, either at the Temple, or Lincoln's
Inn, or Gray's Inn; nor had hi

In [ ]:
import re
import random

sentences = re.split(r'(?<=[.!?])\s+', text)
dataset = []

for i in range(3, len(sentences)-3):
    original = sentences[i]
    story_with_gap = " ".join(sentences[i-3:i]) + " <GAP> " + " ".join(sentences[i+1:i+4])
    dataset.append({"input": story_with_gap, "output": original})

len(dataset)


13811

In [ ]:
from datasets import Dataset
hf_dataset = Dataset.from_list(dataset)
hf_dataset = hf_dataset.train_test_split(test_size=0.1)


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
def preprocess(example):
    inputs = ["fill the gap: " + i for i in example["input"]]
    model_inputs = tokenizer(inputs, truncation=True, padding="max_length", max_length=256)
    labels = tokenizer(example["output"], truncation=True, padding="max_length", max_length=64)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = hf_dataset.map(preprocess, batched=True)


Map:   0%|          | 0/12429 [00:00<?, ? examples/s]

Map:   0%|          | 0/1382 [00:00<?, ? examples/s]

In [ ]:
small_dataset = tokenized_dataset["train"].select(range(1000))
trainer.train_dataset = small_dataset


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="/content/story-gap-model",
    learning_rate=3e-5,
    per_device_train_batch_size=2,
    num_train_epochs=1,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"]
)

trainer.train()


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss


In [ ]:
import os

os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "offline"
os.environ["HF_DISABLE_WANDB"] = "1"


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="/content/story-gap-model",
    learning_rate=3e-5,
    per_device_train_batch_size=2,
    num_train_epochs=1,
    logging_strategy="no",     # <--- disables wandb logging completely
    report_to=[]               # <--- prevents wandb from activating
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"]
)

trainer.train()


In [ ]:
test_text = """
Phileas Fogg left London hurriedly. <GAP> Passepartout followed him nervously,
still unsure about the sudden journey.
"""

inputs = tokenizer("fill the gap: " + test_text, return_tensors="pt")
output = model.generate(inputs["input_ids"], max_length=50)

print(tokenizer.decode(output[0], skip_special_tokens=True))
